In [40]:
import pandas as pd
import janitor
import warnings

warnings.filterwarnings("ignore")

from utilities import process_bl_json_files, pandas_to_tex


FP_US_DOMAINS = "../data/us_gov_domain_list.csv"
FP_US_BL = "../data/us_blacklight_json"
FP_IN_DOMAINS = "../data/in_gov_domain_list.csv"
FP_IN_BL = "../data/in_blacklight_json"
FP_CC_DOMAINS = "../data/common_crawl_sample.csv"
FP_CC_BL = "../data/cc_blacklight_json"

## US

In [2]:
df_us = (
    pd.read_csv(FP_US_DOMAINS)
    .clean_names()
    .rename_column("domain_name", "domain")
)
df_us.head(3)

,domain,domain_type,agency,organization_name,city,state,security_contact_email
0,36thdistrictcourtmi.gov,City,Non-Federal Agency,36th District Court,Detroit,MI,govdnssecurity@36thdistrictcourt.org
1,59dcmi.gov,City,Non-Federal Agency,59th District Court,Walker,MI,soc@walker.city
2,abbevillecitymarshal.gov,City,Non-Federal Agency,Abbeville City Marshal Office,Abbeville,LA,(blank)


In [3]:
bl_labels = {
    "ddg_join_ads": "Ad Trackers",
    "third_party_cookies": "Third-Party Cookies",
    "fb_pixel": "Facebook Pixel",
    "google_analytics": "Google Analytics",
    "key_logging": "Keylogging",
    "session_recording": "Session Recording",
    "canvas_fingerprinting": "Canvas Fingerprinting",
}
bl_measures = list(bl_labels.keys())
bl_measures_al1 = ["".join([var, "_al1"]) for var in bl_measures]
bl_measures_al2 = ["".join([var, "_al2"]) for var in bl_measures]
bl_measures

['ddg_join_ads',
 'third_party_cookies',
 'fb_pixel',
 'google_analytics',
 'key_logging',
 'session_recording',
 'canvas_fingerprinting']

In [4]:
df_us_bl = (
    pd.DataFrame(process_bl_json_files(FP_US_BL))
    .merge(df_us, on="domain", how="left", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)

df_us_bl

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,domain_type,agency,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,fonda-fultonvillewastewater.gov,0,0,0,0,0,0,0,Special district,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
1,johnsoncountyso-ne.gov,0,0,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
2,scatdhhs.gov,2,8,0,0,0,0,0,Tribal,Non-Federal Agency,...,False,False,False,True,True,False,False,False,False,False
3,jeffersontown-ny.gov,0,0,0,0,0,0,0,City,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
4,floridasspeechaudiology.gov,1,0,0,0,0,0,0,State or territory,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7785,ehpsmt.gov,2,1,0,0,0,0,0,School district,Non-Federal Agency,...,False,False,False,True,False,False,False,False,False,False
7786,coopercohealth.gov,3,0,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,True,False,False,False,False,False,False
7787,randolphcountyil.gov,1,4,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,False,True,False,False,False,False,False
7788,boveymn.gov,1,0,0,0,0,0,0,City,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False


In [5]:
df_us_summ = (
    df_us_bl.select_columns(bl_measures)
    .describe(percentiles=[0.25, 0.5, 0.75, 0.9])
    # =====================================================
    # Tidy table and Fromat numbers
    .T.reset_index(names="var")
    .remove_columns("count")
    .applymap(lambda x: f"{x:,.1f}" if isinstance(x, (float, int)) else x)
    # =====================================================
    # how many domains have at least one
    .merge(
        (
            df_us_bl.select_columns(bl_measures_al1)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast1")
            .rename_column("index", "var")
            .replace("_al1", "", regex=True)
        ),
        how="left",
        on="var",
        validate="1:1",
    )
    # =====================================================
    # how many domains have at least two
    .merge(
        (
            df_us_bl.select_columns(bl_measures_al2)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast2")
            .rename_column("index", "var")
            .replace("_al2", "", regex=True)
        ),
        how="left",
        on="var",
        validate="1:1",
    )
    # =====================================================
    .assign(var=lambda df_: df_["var"].replace(bl_labels))
)
pandas_to_tex(
    df_us_summ,
    "../tables/bl_summ_us_domains",
    escape=True,
)
df_us_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,1.7,2.8,0.0,0.0,1.0,2.0,4.0,37.0,71.7%,38.7%
1,Third-Party Cookies,1.2,4.5,0.0,0.0,0.0,1.0,4.0,86.0,26.0%,15.0%
2,Facebook Pixel,0.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,3.0%,0.0%
3,Google Analytics,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.7%,0.0%
4,Keylogging,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.8%,0.0%
5,Session Recording,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.5%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.6%,0.0%


In [6]:
!cat ../tables/bl_summ_us_domains.tex

Ad Trackers & 1.7 & 2.8 & 0.0 & 0.0 & 1.0 & 2.0 & 4.0 & 37.0 & 71.7\% & 38.7\% \\
Third-Party Cookies & 1.2 & 4.5 & 0.0 & 0.0 & 0.0 & 1.0 & 4.0 & 86.0 & 26.0\% & 15.0\% \\
Facebook Pixel & 0.0 & 0.2 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 3.0\% & 0.0\% \\
Google Analytics & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.7\% & 0.0\% \\
Keylogging & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.8\% & 0.0\% \\
Session Recording & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.5\% & 0.0\% \\
Canvas Fingerprinting & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.6\% & 0.0\% \\

## IN

In [34]:
df_in = (
    pd.read_csv(FP_IN_DOMAINS)
    .rename_column("url", "domain")
    .dropna(subset=["domain"])
    .drop_duplicates("domain")
)
df_in.head(3)

,sector,name,domain
0,Agriculture & Cooperation,"State Wetland Authority, Haryana",https://swa.haryana.gov.in/
1,Agriculture & Cooperation,Krishi Vigyan Kendra Knowledge Network,https://kvk.icar.gov.in/
2,Agriculture & Cooperation,Bihar State Election Authority (BSEA),https://bsea.bihar.gov.in/


In [35]:
df_in[df_in.duplicated("domain", keep=False)].sort_values("domain")

,sector,name,domain


In [38]:
df_in_bl = (
    pd.DataFrame(process_bl_json_files(FP_IN_BL))
    .merge(df_in, on="domain", how="left", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)

df_in_bl

Error processing yoga.ayush.gov.in.json: list index out of range
Error processing wbkanyashree.gov.in.json: list index out of range
Error processing rural.assam.gov.in.json: list index out of range
Error processing www.odishaone.gov.in.json: list index out of range
Error processing fcsca.assam.gov.in.json: list index out of range
Error processing rdsdemaharashtra.dgt.gov.in.json: list index out of range
Error processing www.ocac.in.json: list index out of range
Error processing nhm.utl.gov.in.json: list index out of range
Error processing nehrutrophy.nic.in.json: list index out of range
Error processing www.hyderabadpolice.gov.in.json: list index out of range
Error processing cidwestbengal.gov.in.json: list index out of range
Error processing bcasindia.gov.in.json: list index out of range
Error processing www.nmdfc.org.json: list index out of range
Error processing geoportal.natmo.gov.in.json: list index out of range
Error processing asacs.assam.gov.in.json: list index out of range
Err

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,sector,name,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,brns.res.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,ayodhya.cantt.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,ociservices.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,pngrb.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,www.rites.com,3,2,0,0,0,0,0,NaN,NaN,...,False,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,bhavishya.nic.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2471,vizagcustoms.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2472,powerfoundation.org.in,1,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2473,darbhangadivision.bih.nic.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [39]:
df_in_summ = (
    df_in_bl.select_columns(bl_measures)
    .describe(percentiles=[0.25, 0.5, 0.75, 0.9])
    # =====================================================
    # Tidy table and Fromat numbers
    .T.reset_index(names="var")
    .remove_columns("count")
    .applymap(lambda x: f"{x:,.1f}" if isinstance(x, (float, int)) else x)
    # =====================================================
    # how many domains have at least one
    .merge(
        (
            df_in_bl.select_columns(bl_measures_al1)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast1")
            .rename_column("index", "var")
            .replace("_al1", "", regex=True)
        ),
        how="left",
        on="var",
        validate="1:1",
    )
    # =====================================================
    # how many domains have at least two
    .merge(
        (
            df_in_bl.select_columns(bl_measures_al2)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast2")
            .rename_column("index", "var")
            .replace("_al2", "", regex=True)
        ),
        how="left",
        on="var",
        validate="1:1",
    )
    # =====================================================
    .assign(var=lambda df_: df_["var"].replace(bl_labels))
)
# pandas_to_tex(
#     df_in_summ,
#     "../tables/bl_summ_in_domains",
#     escape=True,
# )
df_in_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.7,2.1,0.0,0.0,0.0,1.0,3.0,42.0,28.5%,13.3%
1,Third-Party Cookies,0.5,3.5,0.0,0.0,0.0,0.0,0.0,84.0,8.3%,5.5%
2,Facebook Pixel,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.8%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.2%,0.0%
4,Keylogging,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.9%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.8%,0.0%
